In [22]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, InputLayer, Dropout, Dense, Flatten, Embedding
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

from tensorflow.keras.layers import concatenate
from sklearn.model_selection import train_test_split

In [23]:
df = pd.read_csv('data/affcon_final.csv')

In [7]:
df.columns

Index(['Input.convo_id', 'Input.train_test_val', 'Input.msg_id',
       'Input.timestamp', 'Input.full_text', 'affcon_gamemove',
       'affcon_reasoning', 'affcon_rapport', 'affcon_shareinformation',
       'Input.speaker', 'Input.reply_to', 'Input.speaker_intention',
       'Input.reciever_perception', 'Input.reciever',
       'Input.absolute_message_index', 'Input.relative_message_index',
       'Input.year', 'Input.game_score_speaker', 'Input.game_score_receiver',
       'Input.game_score_delta', 'Input.deception_quadrant', 'Input.num_words',
       'Input.num_characters', 'Input.sno', 'Input.sno1'],
      dtype='object')

In [6]:
df['Input.full_text'] = df['Input.full_text'].fillna(0)
df['Input.full_text'] = df['Input.full_text'].replace('\n','', regex=True)

In [24]:
train, test = train_test_split(df, test_size=0.2)

In [25]:
X_train = train['Input.full_text'].to_list()
y_train_rapport = train['affcon_rapport'].tolist()
y_train_share_information = train['affcon_shareinformation'].tolist()
y_train_reasoning = train['affcon_reasoning'].tolist()
y_train_gamemove = train['affcon_gamemove'].tolist()
y_train_deception_quad = train['Input.deception_quadrant'].tolist()

X_test = test['Input.full_text'].to_list()
y_test_rapport = test['affcon_rapport'].tolist()
y_test_share_information = test['affcon_shareinformation'].tolist()
y_test_reasoning = test['affcon_reasoning'].tolist()
y_test_gamemove = test['affcon_gamemove'].tolist()
y_test_deception_quad = test['Input.deception_quadrant'].tolist()

In [26]:
from transformers import BertModel, BertTokenizer, BertConfig

max_length = 100
configuration = BertConfig()
model = BertModel(configuration)
configuration = model.config
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [27]:
Y_train_rapport_class = to_categorical(y_train_rapport)
Y_train_share_information_class = to_categorical(y_train_share_information)
Y_train_reasoning_class = to_categorical(y_train_reasoning)
Y_train_gamemove_class = to_categorical(y_train_gamemove)
Y_train_deception_class = to_categorical(y_train_deception_quad)

Y_test_rapport_class = to_categorical(y_test_rapport)
Y_test_share_information_class = to_categorical(y_test_share_information)
Y_test_reasoning_class = to_categorical(y_test_reasoning)
Y_test_gamemove_class = to_categorical(y_test_gamemove)
Y_test_deception_class = to_categorical(y_test_deception)

In [28]:
X_train_text = tokenizer(
    text=X_train,
    add_special_tokens=True,
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

X_test_text = tokenizer(
    text=X_test,
    add_special_tokens=True,
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [29]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [32]:
# Game move model
input_ids_in = tf.keras.layers.Input(shape=(max_length,), name='input_token', dtype='int32')

embedding_layer = auto_model(input_ids_in)[0]
cls_token = embedding_layer[:,0,:]
X = tf.keras.layers.BatchNormalization()(cls_token)
X = tf.keras.layers.Dense(192, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(2, activation='softmax')(X)
gamemove_model = tf.keras.Model(inputs=input_ids_in, outputs = X)

gamemove_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
history = gamemove_model.fit(X_train_text['input_ids'], Y_train_gamemove_class, epochs=8, batch_size=8, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = gamemove_model.evaluate(X_test_text['input_ids'], Y_test_gamemove_class, verbose=0)
print(precision, recall, f1_score)

Epoch 1/8


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[8,12,100,100] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node functional_11/tf_bert_for_pre_training_1/bert/encoder/layer_._0/attention/self/Softmax (defined at C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\transformers\modeling_tf_bert.py:262) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[gradient_tape/functional_11/tf_bert_for_pre_training_1/bert/embeddings/add/Sum/_30]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[8,12,100,100] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node functional_11

In [17]:
# Reasoning model 
input_ids_in = tf.keras.layers.Input(shape=(max_length,), dtype='int32')

embedding_layer = auto_model(input_ids_in)[0]
cls_token = embedding_layer[:,0,:]
X = tf.keras.layers.BatchNormalization()(cls_token)
X = tf.keras.layers.Dense(192, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(2, activation='softmax')(X)
reasoning_model = tf.keras.Model(inputs=input_ids_in, outputs = X)

reasoning_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
history = reasoning_model.fit(X_train_text['input_ids'], Y_train_rapport_class, epochs=8, batch_size=8, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = reasoning_model.evaluate(X_test_text['input_ids'], Y_test_rapport_class, verbose=0)
print(precision, recall, f1_score)

2518/2518 [==============================] - 357s 142ms/step - loss: 4.7668 - acc: 0.6881 - f1_m: 0.6881 - precision_m: 0.6881 - recall_m: 0.6881 - val_loss: 3.6729 - val_acc: 0.7605 - val_f1_m: 0.7607 - val_precision_m: 0.7607 - val_recall_m: 0.7607
0.7646253705024719 0.7646253705024719 0.7646253108978271


In [18]:
# Share information model 
input_ids_in = tf.keras.layers.Input(shape=(max_length,), dtype='int32')

embedding_layer = auto_model(input_ids_in)[0]
cls_token = embedding_layer[:,0,:]
X = tf.keras.layers.BatchNormalization()(cls_token)
X = tf.keras.layers.Dense(192, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(2, activation='softmax')(X)
shareinfo_model = tf.keras.Model(inputs=input_ids_in, outputs = X)

shareinfo_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
history = shareinfo_model.fit(X_train_text['input_ids'], Y_train_share_information_class, epochs=8, batch_size=8, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = reasoning_model.evaluate(X_test_text['input_ids'], Y_test_share_information_class, verbose=0)
print(precision, recall, f1_score)

2518/2518 [==============================] - 358s 142ms/step - loss: 7.4520 - acc: 0.5132 - f1_m: 0.5132 - precision_m: 0.5132 - recall_m: 0.5132 - val_loss: 7.0838 - val_acc: 0.5381 - val_f1_m: 0.5377 - val_precision_m: 0.5377 - val_recall_m: 0.5377
0.5111532211303711 0.5111532211303711 0.5111531615257263


In [ ]:
# Rapport model 
input_ids_in = tf.keras.layers.Input(shape=(max_length,), dtype='int32')

embedding_layer = auto_model(input_ids_in)[0]
cls_token = embedding_layer[:,0,:]
X = tf.keras.layers.BatchNormalization()(cls_token)
X = tf.keras.layers.Dense(192, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(2, activation='softmax')(X)
rapport_model = tf.keras.Model(inputs=input_ids_in, outputs = X)

rapport_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
history = rapport_model.fit(X_train_text['input_ids'], Y_train_rapport_class, epochs=8, batch_size=8, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = reasoning_model.evaluate(X_test_text['input_ids'], Y_test_rapport_class, verbose=0)
print(precision, recall, f1_score)

In [19]:
# game move + reasoning - shareinfo

from keras.layers import * 

commonInput = tf.keras.layers.Input(shape=(max_length,))

out1 = gamemove_model(commonInput)    
out2 = reasoning_model(commonInput)   
out3 = shareinfo_model(commonInput)
mergedTwo = tf.keras.layers.Add()([out1,out2])
mergedOut = tf.keras.layers.Subtract()([mergedTwo, out3])

mergedOut = tf.keras.layers.Flatten()(mergedOut)    
mergedOut = tf.keras.layers.Dense(256, activation='relu')(mergedOut)
mergedOut = tf.keras.layers.Dropout(.5)(mergedOut)
mergedOut = tf.keras.layers.Dense(128, activation='relu')(mergedOut)
mergedOut = tf.keras.layers.Dropout(.35)(mergedOut)
mergedOut = tf.keras.layers.Dense(2, activation='softmax')(mergedOut)  #Cuz binary

mergedModel = tf.keras.Model(commonInput, mergedOut)
mergedModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
mergedModel.fit(X_train_text['input_ids'], Y_train_rapport_class, epochs=8, batch_size=8, 
                    validation_split=0.2, callbacks=[callback])

In [ ]:
# game move + reasoning + shareinfo

from keras.layers import * 

commonInput = tf.keras.layers.Input(shape=(max_length,))

out1 = gamemove_model(commonInput)    
out2 = reasoning_model(commonInput)   
out3 = shareinfo_model(commonInput)
mergedOut = tf.keras.layers.Add()([out1,out2,out3])

mergedOut = tf.keras.layers.Flatten()(mergedOut)    
mergedOut = tf.keras.layers.Dense(256, activation='relu')(mergedOut)
mergedOut = tf.keras.layers.Dropout(.5)(mergedOut)
mergedOut = tf.keras.layers.Dense(128, activation='relu')(mergedOut)
mergedOut = tf.keras.layers.Dropout(.35)(mergedOut)
mergedOut = tf.keras.layers.Dense(2, activation='softmax')(mergedOut)  #Cuz binary

#mergedModel = Model([model1.input,model2.input], mergedOut)
mergedModel = tf.keras.Model(commonInput, mergedOut)
mergedModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
mergedModel.fit(X_train_text['input_ids'], Y_train_rapport_class, epochs=8, batch_size=8, 
                    validation_split=0.2, callbacks=[callback])

In [ ]:
# game move + reasoning + shareinfo + rapport

from keras.layers import * 

commonInput = tf.keras.layers.Input(shape=(max_length,))

out1 = gamemove_model(commonInput)    
out2 = reasoning_model(commonInput)   
out3 = shareinfo_model(commonInput)
out4 = rapport_model(commonInput)
mergedOut = tf.keras.layers.Add()([out1,out2,out3,out4])

mergedOut = tf.keras.layers.Flatten()(mergedOut)    
mergedOut = tf.keras.layers.Dense(256, activation='relu')(mergedOut)
mergedOut = tf.keras.layers.Dropout(.5)(mergedOut)
mergedOut = tf.keras.layers.Dense(128, activation='relu')(mergedOut)
mergedOut = tf.keras.layers.Dropout(.35)(mergedOut)
mergedOut = tf.keras.layers.Dense(2, activation='softmax')(mergedOut)  #Cuz binary

#mergedModel = Model([model1.input,model2.input], mergedOut)
mergedModel = tf.keras.Model(commonInput, mergedOut)
mergedModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
mergedModel.fit(X_train_text['input_ids'], Y_test_deception_class, epochs=8, batch_size=8, 
                    validation_split=0.2, callbacks=[callback])

In [21]:
X_text_to_predict = tokenizer(
    text="hello how are you? I love you",
    add_special_tokens=True,
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

mergedModel.predict(X_text_to_predict['input_ids'])

array([[0.41065216, 0.5893479 ]], dtype=float32)

In [ ]:
# Other merge strategies https://github.com/A2Zadeh/TensorFusionNetwork/blob/master/tf_mosi.py